# Capstone Project: Classifying clinically actionable genetic mutations

***

## Notebook 4: Alternative Models

This notebook contains code to explore alternative models to address the classification problem.

### Contents

- [Importing of Libraries](#Importing-of-Libraries)
- [Data Import](#Data-Import)

## Importing of Libraries

In [1]:
import pandas as pd
import numpy as np

from tabulate import tabulate
from gensim.models.word2vec import Word2Vec
from collections import Counter, defaultdict

TRAIN_SET_PATH = "../assets/train_prep.csv"

GLOVE_6B_50D_PATH = "../assets/glove.6B.50d.txt"
GLOVE_6B_300D_PATH = "../assets/glove.6B.300d.txt"
encoding="utf-8"

BERT_INIT_CHKPNT = "../assets/cased_L-12_H-768_A-12/bert.model.ckpt.index"
BERT_VOCAB = "../assets/cased_L-12_H-768_A-12/vocab.txt"
BERT_CONFIG = "../assets/cased_L-12_H-768_A-12/bert_config.json"

from sklearn import linear_model, metrics, svm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV,\
    cross_val_score, RandomizedSearchCV, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, label_binarize, MinMaxScaler
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score, classification_report, f1_score
from sklearn.multiclass import OneVsRestClassifier

from imblearn.over_sampling import SMOTE

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle
plt.style.use('fivethirtyeight')

import time
import datetime
import pickle
import os

import nltk
from nltk.tokenize import RegexpTokenizer
import regex as re
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU, LSTM, Embedding, SpatialDropout1D, Flatten
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import pad_sequences

import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
# import tensorflow as tf
import tensorflow.compat.v1 as tf
#To make tf 2.0 compatible with tf1.0 code, we disable the tf2.0 functionalities
tf.disable_eager_execution()
import tensorflow_hub as hub

from wordcloud import WordCloud

import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Initialise random seeed for more consistent results
from numpy.random import seed
seed(42)

Using TensorFlow backend.


## Data Import

In [2]:
# Import 'train_prep' and 'test_prep' datasets
# We use the 'keep_default_na' option to False to ensure that pandas does not re-introduce missing values
train = pd.read_csv("../assets/train_prep.csv", keep_default_na=False)
test = pd.read_csv("../assets/test_prep.csv", keep_default_na=False)

In [3]:
train.shape, test.shape

((3321, 4325), (986, 4324))

In [4]:
train.head(2)

,id,class,text,gene_ABCB11,gene_ABCC6,gene_ABL1,gene_ACVR1,gene_ADAMTS13,gene_ADGRG1,gene_AGO2,...,variation_YAP1-TFE3 Fusion,variation_YWHAE-ROS1 Fusion,variation_ZC3H7B-BCOR Fusion,variation_ZNF198-FGFR1 Fusion,variation_null1313Y,variation_null189Y,variation_null262Q,variation_null267R,variation_null399R,variation_p61BRAF
0,0,1,cyclin dependent kinase cdks regulate variety ...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,abstract background non small lung nsclc heter...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
test.head(2)

,id,text,gene_ABCB11,gene_ABCC6,gene_ABL1,gene_ACVR1,gene_ADAMTS13,gene_ADGRG1,gene_AGO2,gene_AGXT,...,variation_YAP1-TFE3 Fusion,variation_YWHAE-ROS1 Fusion,variation_ZC3H7B-BCOR Fusion,variation_ZNF198-FGFR1 Fusion,variation_null1313Y,variation_null189Y,variation_null262Q,variation_null267R,variation_null399R,variation_p61BRAF
0,1,incidence breast increase china recent decade ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,unselected series colorectal carcinoma stratif...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Preparation for Modelling

In [6]:
# Splitting of data into Predictor (X) and Target (y) Dataframes
X = train[[i for i in train.columns if i not in ['id', 'class']]]
y = train['class']

In [7]:
X.shape, y.shape

((3321, 4323), (3321,))

In [8]:
# Save copies of the original X and y dataframes so that we can try different word embeddings as needed
X_original = X.copy()
y_original = y.copy()

In [9]:
X_test = test.drop(['id'], axis=1)

In [10]:
X_test.shape

(986, 4323)

In [11]:
%%time
# For convenience, we create an array called 'X_all_text that contains
# all the text from our training set, across all samples
X_all_text = []
for text in X['text']:
    X_all_text.append(text.split())
X_all_text = np.array(X_all_text)

Wall time: 2.77 s


In [12]:
X_all_text.shape

(3321,)

In [13]:
# Create a set of all the unique words from our taining set, across all samples
X_unique_words = set()
for row in range(len(X_all_text)):
    X_unique_words.update(X_all_text[row])

In [14]:
len(X_unique_words)

81312

In [15]:
max_phrase_len = 0
for row in range(len(X_all_text)):
    if len(X_all_text[row]) > max_phrase_len:
        max_phrase_len = len(X_all_text[row])

In [16]:
max_phrase_len

44019

## Exploring alternative word embeddings

In this notebook, we evaluate various word embeddings other than the ones created using the standard TfidfVectorizer as used in the baseline model.

### Evaluation Approach

To manage the overall processing power needed to find the best alternative model, we seek to find the best alternative model based on the folowing steps:
1. Find an alternative set of word embeddings but use a fixed classifier (Extra Trees Classifier) to measure their performance against our validation dataset
2. Find the best classifier for our chosen word embeddings

For **Step 1** above, we explore three static word embeddings to see if we can get better classification results:
- Global Vectors for Word Representation (GloVe) pre-trained word embeddings created by the [Stanford Natural Language Processing Group](https://nlp.stanford.edu/) and available at https://nlp.stanford.edu/projects/glove/:
    - A smaller set of GloVe embeddings (which we call **'glove_small'**) that are based on based on Wikipedia 2015 and Gigaword 5th Edition (https://catalog.ldc.upenn.edu/LDC2011T07)
    - A larger set of GloVe embeddings (which we call **'glove_big'**) that are based on Common Crawl (https://commoncrawl.org/)
- Our own word embeddings created by training Word2Vec (from nltk) on all the given text in the training dataset, which we call **'w2v'**.

For **Step 2** above, we evaluate the performance of a Forward Neural Network (FNN) and an Extra Trees Classifier. For both classifiers, we use RandomisedSearchCV to find the set of optimal parameters to give the best cross-validated score based on the training dataset.

### Loading of pre-trained word embeddings based on GloVe

We now load the two pre-trained word embeddings (based on GloVe) and store them as dictionary objects 'glove_small' and 'glove_big'.

In [17]:
%%time
import struct 

# glove_small keys are bounded by what is common to both the X_all_text and the glove file
glove_small = {}
all_words = set(w for words in X_all_text for w in words)
with open(GLOVE_6B_50D_PATH, "rb") as infile:
    for line in infile:
        parts = line.split()
        word = parts[0].decode(encoding)
        if (word in all_words):
            nums=np.array(parts[1:], dtype=np.float32)
            glove_small[word] = nums

# glove_big keys are bounded by what is common to both the X_all_text and the glove file
glove_big = {}
with open(GLOVE_6B_300D_PATH, "rb") as infile:
    for line in infile:
        parts = line.split()
        word = parts[0].decode(encoding)
        if word in all_words:
            nums=np.array(parts[1:], dtype=np.float32)
            glove_big[word] = nums

Wall time: 13.8 s


### Loading of ELMo word embeddings based on training dataset vocabulary

We load the ELMo module (version 3) from TensorFlow Hub. The ELMo module computes contextualized word representations using character-based word representations and bidirectional LSTMs. 

The module supports inputs both in the form of raw text strings or tokenized text strings. It outputs fixed embeddings at each LSTM layer, a learnable aggregation of the 3 layers, and a fixed mean-pooled vector representation of the input.

In [ ]:
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

We define a function that will extract the ELMo vectors of all the words in a single sample and take their mean.

In [ ]:
def elmo_vectors(x):
    embeddings = elmo(x, signature="default", as_dict=True)["elmo"]
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # return average of ELMo features
        return sess.run(tf.reduce_mean(embeddings,1))

We now call the function for each sample text in the X_all_text list we have created earlier. Due to memory constraints, we cannot store the entire set of ELMo vectors in memory. Instead, we write them to disk using the Pickle serialisation library using the most efficient Pickle serialisation protocol available. 

In [ ]:
%%time
num_rows = len(X_all_text)
block_size = 500
first_start = time.time()
for row in range(num_rows):
    row_start = time.time()
    df = pd.DataFrame(elmo_vectors(X_all_text[row]))
    output_file_name = "../assets/elmo_vectors_" + str((row//block_size)*block_size) + ".csv"
    df.to_csv(output_file_name, mode='a')
    print ("Row {} of {} appended, duration (h:m:s): {}, total elapsed time (h:m:s): {}".format(row+1, num_rows+1,
               str(datetime.timedelta(seconds=time.time()-row_start)),
               str(datetime.timedelta(seconds=time.time()-first_start))))

### Creation of Word2Vec word embeddings based on training dataset vocabulary

We now create our own word embeddings using Word2Vec from nltk, trained using all the text in the training dataset.

In [18]:
%%time
model = Word2Vec(X_all_text, size=100, window=5, min_count=5, workers=8)
w2v = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}

Wall time: 1min 6s


### Defining new Vectorizers

We define two word embedding vectorizers - as variations of the CountVectorizer and TfidfVectorizer - that take a word-to-vector mapping and vectorises the text by taking the **mean** of all the 'glove_small' vectors corresponding to individual words.

In [19]:
class MeanEmbeddingVectorizer(object):
    
    # Creating a new MeanEmbeddingVectorizer object with input parameter of 'word2vec'
    # will create a new array of size equal to the number of words contained in word2vec
    # that are also present in the glove_small dictionary
    def __init__(self, word2vec):
        self.word2vec = word2vec
        if len(word2vec)>0:
            self.dim=len(word2vec[next(iter(glove_small))])
        else:
            self.dim=0
            
    # The 'fit' method simply returns the input parameter 'word2vec' itself
    def fit(self, X, y):
        return self 

    # The 'transform' method returns an array consisting of the mean of all
    # 'glove_small" vectors that correspond to individual words in the input parameter 'X'
    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

# TfidfVectorizer is a variation of the Term Frequency - Inverse Document Frequency (Tfidf) Vectorizer
class TfidfEmbeddingVectorizer(object):
    
    # Creating a new TfidfEmbeddingVectorizer object with input parameter of 'word2vec'
    # will create an array of size eequal to the number of words contained in word2vec
    # that are also present in the glove_small dictionary
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        if len(word2vec)>0:
            self.dim=len(word2vec[next(iter(glove_small))])
        else:
            self.dim=0
        
    # The 'fit' method updates the word weights attribute based on the
    # output of the standard TfidfVectorizer
    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf, 
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])
    
        return self
    
    # The 'transform' method returns an array consisting of the mean of all 'glove_small'
    # vectors (adjusted by the Tfidf weights) that correspond to individual words
    # in the input parameter 'X'
    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

## Finding the best combination of word embeddings and vectorizer

We create multiple pipelines that consist of different combinations of the vectorizers (defined above) and the word embeddings, coupled with the Extra Trees classifier. The Extra Trees Classifier is chosen based on its generally good performance; it produces less variance but may cause an increase in bias, compared to a Random Forest classifier.

The different pipelines are as follows:
- mev_w2v: MeanEmbeddingVectorizer based on our own 'w2v' word vectors created using Word2Vec on the words in the training dataset
- tev_w2v: TfidfEmbeddingVectorizer based on our own 'w2v word vectors
- mev_glove_small: MeanEmbeddingVectorizer based on the 'glove_small' word vectors
- tev_glove_small: TfidfEmbeddingVectorizer based on the 'glove_small' word vectors
- mev_glove_big: MeanEmbeddingVectorizer based on the 'glove_big' word vectors
- tev_glove_big: TfidfEmbeddingVectorizer based on the 'glove_big' word vectors

In [ ]:
# Create the various pipelines for evaluation
mev_w2v = Pipeline([("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=200, verbose=1, random_state=42, n_jobs=-1))])
tev_w2v = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=200, verbose=1, random_state=42, n_jobs=-1))])
mev_glove_small = Pipeline([("glove vectorizer", MeanEmbeddingVectorizer(glove_small)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=200, verbose=1, random_state=42, n_jobs=-1))])
tev_glove_small = Pipeline([("glove vectorizer", TfidfEmbeddingVectorizer(glove_small)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=200, verbose=1, random_state=42, n_jobs=-1))])
mev_glove_big = Pipeline([("glove vectorizer", MeanEmbeddingVectorizer(glove_big)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=200, verbose=1, random_state=42, n_jobs=-1))])
tev_glove_big = Pipeline([("glove vectorizer", TfidfEmbeddingVectorizer(glove_big)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=200, verbose=1, random_state=42, n_jobs=-1))])

We now obtain the 3-fold cross-validation accuracy scores for each pipeline based on the labels 'y' (i.e. variation classes), and sort them in descending order of cross-validation accuracy score.

In [ ]:
%%time
all_models = [
    ("mev_w2v", mev_w2v),
    ("tev_w2v", tev_w2v),
    ("mev_glove_small", mev_glove_small),
    ("tev_glove_small", tev_glove_small),
    ("mev_glove_big", mev_glove_big),
    ("tev_glove_big", tev_glove_big)
]

unsorted_scores = [(name, cross_val_score(model, X_all_text, y, cv=3, verbose=1, n_jobs=4).mean())
                   for name, model in all_models]
scores = sorted(unsorted_scores, key=lambda x: -x[1])
print (tabulate(scores, floatfmt=".4f", headers=("Model", 'Cross-Validation Accuracy Score')))

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x=[name for name, _ in scores], y=[score for _, score in scores]);
plt.title("Cross-validated Accuracy Score on Training Dataset")
plt.xlabel("Pipeline");

<div class="alert alert-block alert-info">
As shown above, the <b>Mean Embedding Vectorizer</b> (mev) performs best together with the <b>Word2Vec</b> (w2v) word embedding based on the training dataset vocabulary, as it scores the highest in terms of the cross-validated accuracy score.<br>

We now evaluate two options for identifying the right classifier - a Forward Neural Network (FNN) and an Extra Trees Classifier.
</div>

## Finding the best classifier

### Creation of (Inner) Training and Validation Datasets

From our single training data set (X and y) we will create two separate datasets:
- (Inner) Training Dataset: this will be used to train our models (this will take 75% of the original training dataset)
- Validation Dataset: this will be used to validate our trained models (e.g. check for overfitting) (this will take 25% of our total 'posts' dataset

To create our datasets, we use train_test_split with the stratify option to ensure a consistent mix of values for the target feature within the created datasets.

In [20]:
# Restore the original predictor and target dataframes
X = X_original.copy()
y = y_original.copy()

In [21]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42, stratify=y)

In [22]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((2490, 4323), (2490,), (831, 4323), (831,))

In [23]:
# Reset the indices to prevent spurious rows from appearing later during merging
X_train.reset_index(inplace=True, drop=True)
X_val.reset_index(inplace=True, drop=True)

### Generation of word embeddings

In [24]:
# Instantiate a Mean Embedding Vectorizer object using the Word2Vec word embeddings
mev = MeanEmbeddingVectorizer(w2v)

In [25]:
%%time
X_train_mev = mev.fit(X_train['text'], y_train)
X_train_mev = mev.transform(X_train['text'])
X_val_mev = mev.transform(X_val['text'])
X_test_mev = mev.transform(X_test['text'])

Wall time: 57 s


In [26]:
X_train_mev.shape, X_val_mev.shape, X_test_mev.shape

((2490, 100), (831, 100), (986, 100))

In [27]:
X_train_mev_df = pd.DataFrame(X_train_mev)
X_val_mev_df = pd.DataFrame(X_val_mev)
X_test_mev_df = pd.DataFrame(X_test_mev)

In [28]:
X_train_mev_df.shape, X_val_mev_df.shape, X_test_mev_df.shape

((2490, 100), (831, 100), (986, 100))

In [29]:
X_train_mev_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.359210,0.258085,-2.380271,3.033347,-0.951150,0.996158,1.210566,-0.556969,-0.558642,1.742598,...,1.752876,1.736327,-0.454960,-0.892684,-0.811404,-0.311803,-0.496243,-0.085654,-1.614565,-0.033796
1,-0.360406,0.177359,-2.307191,3.028348,-1.091883,0.980033,1.181806,-0.658525,-0.477435,1.643556,...,1.931653,1.697055,-0.454070,-0.889624,-0.859909,-0.282633,-0.520913,-0.129737,-1.685553,-0.102000
2,-0.360798,0.205959,-2.341243,3.064570,-1.003718,0.961549,1.219805,-0.707036,-0.556856,1.690990,...,1.839368,1.762386,-0.491441,-0.871915,-0.830880,-0.273453,-0.539925,-0.099585,-1.686311,-0.113099
3,-0.369357,0.203926,-2.286042,2.951351,-1.042583,0.910224,1.142366,-0.662081,-0.495575,1.618607,...,1.819690,1.756825,-0.502824,-0.922658,-0.802987,-0.280596,-0.526998,-0.117783,-1.693253,-0.082380
4,-0.375127,0.172580,-2.325269,3.013671,-1.009086,0.915576,1.192056,-0.614665,-0.532230,1.686688,...,1.845610,1.721992,-0.448528,-0.906437,-0.806377,-0.295361,-0.529862,-0.119363,-1.662606,-0.043600


### Combining word vectors with dummy columns

In [30]:
# Concatenate the components parts of the dataframe
X_train = pd.concat([X_train, X_train_mev_df], axis=1)
X_val = pd.concat([X_val, X_val_mev_df], axis=1)
X_test = pd.concat([X_test, X_test_mev_df], axis=1)

In [31]:
X_train.drop(columns=['text'], inplace=True)
X_val.drop(columns=['text'], inplace=True)
X_test.drop(columns=['text'], inplace=True)

In [32]:
X_train.shape, X_val.shape, X_test.shape

((2490, 4422), (831, 4422), (986, 4422))

In [33]:
X_train.head()

,gene_ABCB11,gene_ABCC6,gene_ABL1,gene_ACVR1,gene_ADAMTS13,gene_ADGRG1,gene_AGO2,gene_AGXT,gene_AKAP9,gene_AKT1,...,90,91,92,93,94,95,96,97,98,99
0,0,0,0,0,0,0,0,0,0,0,...,1.752876,1.736327,-0.454960,-0.892684,-0.811404,-0.311803,-0.496243,-0.085654,-1.614565,-0.033796
1,0,0,0,0,0,0,0,0,0,0,...,1.931653,1.697055,-0.454070,-0.889624,-0.859909,-0.282633,-0.520913,-0.129737,-1.685553,-0.102000
2,0,0,0,0,0,0,0,0,0,0,...,1.839368,1.762386,-0.491441,-0.871915,-0.830880,-0.273453,-0.539925,-0.099585,-1.686311,-0.113099
3,0,0,0,0,0,0,0,0,0,0,...,1.819690,1.756825,-0.502824,-0.922658,-0.802987,-0.280596,-0.526998,-0.117783,-1.693253,-0.082380
4,0,0,0,0,0,0,0,0,0,0,...,1.845610,1.721992,-0.448528,-0.906437,-0.806377,-0.295361,-0.529862,-0.119363,-1.662606,-0.043600


### Handling of imbalanced classes

In [34]:
y_train.value_counts(normalize=True)

7    0.287149
4    0.206426
1    0.171084
2    0.136145
6    0.082731
5    0.072691
3    0.026908
9    0.011245
8    0.005622
Name: class, dtype: float64

In [35]:
y_train.value_counts()

7    715
4    514
1    426
2    339
6    206
5    181
3     67
9     28
8     14
Name: class, dtype: int64

We follow the same oversampling strategy as per the baseline model -- i.e. to use SMOTE to oversample the 3 most infrequent classes to be 100 samples each.

In [36]:
# Instantiate a SMOTE object to oversample minority classes
sm = SMOTE(random_state=42, sampling_strategy={3:100, 9:100, 8:100})

In [37]:
%%time
X_train, y_train = sm.fit_sample(X_train, y_train)

Wall time: 1.46 s


In [38]:
X_train.shape, y_train.shape

((2681, 4422), (2681,))

In [39]:
# Verify that we now have 181 samples for classes '5', '3' and '8'.
y_train.value_counts()

7    715
4    514
1    426
2    339
6    206
5    181
9    100
3    100
8    100
Name: class, dtype: int64

### Scaling the data

In [40]:
ms = MinMaxScaler()

In [41]:
ms.fit(pd.concat([X_train, X_val])) # we fit the StandardScaler based on all our training and validation data
X_train = ms.transform(X_train)
X_val = ms.transform(X_val)
X_test = ms.transform(X_test)

## Randomised Search for optimal classifier parameters

To manage the total time and resources used to tune the classifier parameters, we use the RandomizedSearchCV to randomly select parameters from the specified ranges of parameters to give the best cross-validated accuracy score on the training dataset, with a maximum of 10 iterations. We specify the range of parameters for each classifer based on experience and past results of running the RandomizedSearchCV.

We select the best classifier as the one with the highest accuracy score on the **validation dataset**.

In [42]:
def model_func(layer_one_neurons=3000, layer_one_dropout=0.2,
               layer_two_neurons=2000, layer_two_dropout=0.2, opt_learning_rate=0.001):

    model = Sequential()

    model.add(Dense(layer_one_neurons, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dropout(layer_one_dropout))

    model.add(Dense(layer_two_neurons, activation='relu'))
    model.add(Dropout(layer_two_dropout))

    model.add(Dense(9,activation='softmax'))

    Ad = optimizers.Adam(learning_rate=opt_learning_rate,
                         beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(loss='categorical_crossentropy', optimizer=Ad, metrics=['accuracy'])
    return model

In [43]:
# We have selected the models below for modelling purposes.
estimators = {
    'nn': KerasClassifier(build_fn=model_func, epochs=20, verbose=2),
    'svm': SVC(random_state=42),
    'lr': LogisticRegression(random_state=42),
    'etree': ExtraTreesClassifier(random_state=42),
    'ada': AdaBoostClassifier(random_state=42),
    'knn': KNeighborsClassifier(),
    'rf': RandomForestClassifier(random_state=42),
    'dtree': DecisionTreeClassifier(random_state=42),
    'mnb': MultinomialNB()
}.items()

In [44]:
params = {
    'nn': {
        'nn__layer_one_neurons': [2000, 3000, 4000],
        'nn__layer_one_dropout': [0.1, 0.2, 0.3],
        'nn__layer_two_neurons' : [1000, 2000, 3000],
        'nn__layer_two_dropout': [0.1, 0.2, 0.3],
        'nn__opt_learning_rate' : [0.001, 0.0001]
    },
    'svm': {
        'svm__C': [0.1, 1.0, 10],
        'svm__kernel': ['linear','poly', 'rbf', 'sigmoid'],
        'svm__class_weight': ['balanced'] # 'balanced' will help to deal with our imbalanced classes
    },
    'lr': {
        # 'liblinear' solver has been excluded as a potential solver as it cannot learn a true multinomial
        # (multiclass) model; instead, the optimization problem is decomposed in a “one-vs-rest”
        # fashion so separate binary classifiers are trained for all classes.
        # 'lbfgs' solver has also been excluded as it fails to converge through past attempts
        'lr__solver': ['sag','saga'], 
        'lr__penalty': ['l1', 'l2'],
        'lr__C': [0.1, 1.0, 10],
        'lr__max_iter': [50], # limit the max. no. of iterations to 50 to speed up processing
        'lr__multi_class': ['multinomial'],
        'lr__class_weight': ['balanced'] # 'balanced' will help to deal with our imbalanced classes
    },
    'etree': {
        'etree__max_features': ['auto', 'sqrt', 'log2', None],
        'etree__min_samples_split': [4, 6, 8],
        'etree__min_samples_leaf': [2, 3, 4],
        'etree__class_weight': ['balanced'] # 'balanced' will help to deal with our imbalanced classes
    },
    'ada': {
        'ada__n_estimators': [50, 100, 150],
        'ada__learning_rate': [1, 1.5, 2]
    },
    'knn': {
        'knn__n_neighbors': [3, 5, 7]
    },
    'rf': {
        'rf__n_estimators': [100, 200, 300],
        'rf__class_weight': ['balanced'], # 'balanced' will help to deal with our imbalanced classes
        'rf__min_samples_split': [5, 10, 15],
        'rf__min_samples_leaf': [2, 3, 4]
    },
    'dtree': {
        'dtree__max_features': ['auto', 'sqrt', 'log2', None],
        'dtree__min_samples_split': [4, 6, 8],
        'dtree__min_samples_leaf': [2, 3, 4],
        'dtree__class_weight': ['balanced'] # 'balanced' will help to deal with our imbalanced classes
    },
    'mnb': {
        'mnb__alpha': [0, 0.5, 1.0],
        'mnb__fit_prior': [True, False]  
    }
}

We now use RandomizedSearchCV to select the optimal parameters for each classifier that produces the best 3-fold cross-validated mean accuracy score based on the training dataset.

In [ ]:
%%time
# initialise empty lists to store information later
models = []
parameters = []
best_score = []
train_bal_f1_score = []
val_val_f1_score = []
train_bal_accuracy = []
val_bal_accuracy = []

for k,v in estimators:
    start = time.time()
    pipe = Pipeline([(k,v)])
    param = params[k]
    randomsearch = RandomizedSearchCV(
        n_iter = 10, # we set a max. of 10 iterations
        estimator = pipe,
        random_state = 42,
        param_distributions = param,
        verbose = 1,
        cv = 3, # 3-fold cross-validation
        n_jobs=4, # 4 concurrent jobs
        return_train_score= True,
        scoring = 'accuracy' # use best cross-validation accuracy score
    )

    now = datetime.datetime.now()
    print ("Model: ", k)
    print ("Started fitting at: {}".format(now.strftime("%Y-%m-%d %H:%M:%S")))
    
    randomsearch.fit(X_train, y_train)
    
    model = randomsearch.best_estimator_
    cv_score = randomsearch.cv_results_
    best_params = randomsearch.best_params_

    # predict y
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    
    # print results
    print ("Fitting duration (h:m:s): {}".format(str(datetime.timedelta(seconds=time.time()-start))))
    print ("Best parameters:", best_params)
    print ("Best cross-validation accuracy score:", randomsearch.best_score_)
    print ("Training set balanced F1 score:", f1_score(y_train, y_train_pred, average='weighted'))
    print ("Validation set balanced F1 score:", f1_score(y_val, y_val_pred, average='weighted'))
    print ("Training set balanced accuracy:", balanced_accuracy_score(y_train,y_train_pred))
    print ("Validation set balanced accuracy:", balanced_accuracy_score(y_val,y_val_pred))
    print ("")
    
    # append info to list
    models.append(k)
    best_score.append(randomsearch.best_score_)
    parameters.append(best_params)
    train_bal_f1_score.append(f1_score(y_train,y_train_pred, average='weighted'))
    val_bal_f1_score.append(f1_score(y_val,y_val_pred, average='weighted'))
    train_bal_accuracy.append(balanced_accuracy_score(y_train,y_train_pred))
    val_bal_accuracy.append(balanced_accuracy_score(y_val,y_val_pred))

Model:  nn
Started fitting at: 2020-04-06 19:58:11
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


## Confirmation of Alternative Model

In [ ]:
# Produce a summary table of the tuned classifiers
summary = pd.DataFrame({
    'model': models,
    'parameters': parameters,
    'Best cross-validation accuracy score': best_score,
    'Training set balanced F1 score': train_bal_f1_score,
    'Validation set balanced F1 score': val_bal_f1_score,
    'Training set balanced accuracy': train_bal_accuracy,
    'Validation set balanced accuracy': val_bal_accuracy
    })

pd.set_option('display.max_colwidth', None)
summary.sort_values('Validation set balanced accuracy', ascending=False).reset_index(drop=True)

<div class="alert alert-block alert-info">
    
Comparing the validation dataset accuracy, we have:
    <ul>
        <li> `0.58845` for FNN Classifier</li>
        <li> `0.64862` for the Extra Trees Classifier</li>
    </ul>
<br>
    Based on the highest score above, our best alternative model is therefore the <b>Extra Trees classifier</b> trained on <b>Word2Vec word embeddings</b> using the <b>Mean Embedding Vectorizer</b>.

We note that the validation (dataset) accuracy of this model (`0.64862`) is <b>lower</b> than the baseline model (`0.66787`), which suggests that the alternative model is <b>more overfitted</b> than the baseline model. We now generate predictions based on the chosen alternative model so that we can obtain the Kaggle score in notebook 5.
</div>

## Further Exploration of Alternative Model

In [ ]:
# We instantiate the best classifier based on the best parameters found above
alternative_clf = ExtraTreesClassifier(verbose=1, n_jobs=4, random_state=42, n_estimators=250,
                                       min_samples_split=4, min_samples_leaf=2, max_features=None)

In [ ]:
%%time
# Fit the best classifier on the training dataset
alternative_clf.fit(X_train, y_train)

### ROC Curve & Metrics

In [ ]:
# Generate predictions for the validation data based on our baseline model
y_val_pred = alternative_clf.predict(X_val)

In [ ]:
# Binarize the output
y_train_binarized = label_binarize(y_train, classes=list(np.unique(y)))
y_val_pred_binarized = label_binarize(y_val_pred, classes=list(np.unique(y)))
n_classes = len(np.unique(y))

In [ ]:
%%time
now = datetime.datetime.now()
print ("Started fitting at: {}".format(now.strftime("%Y-%m-%d %H:%M:%S")))
# We obtain the distances of each sample from the decision boundary for each class
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True,
                                 random_state=42, verbose=1), n_jobs=2)
y_score = classifier.fit(X_train, y_train_binarized).decision_function(X_val)

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    # we compare our predicted labels for the validation dataset and the actual validation dataset labels
    fpr[i], tpr[i], _ = roc_curve(y_val_pred_binarized[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_val_pred_binarized.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

We plot ROC curves for all the 9 classes.

In [ ]:
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(figsize=(12,8))
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

lw=2
colors = cycle(['rosybrown', 'firebrick', 'sienna', 'olivedrab', 'darkgreen',\
                'lightseagreen', 'darkturquoise', 'b', 'darkorange'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i+1, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Multiple Classes')
plt.legend(loc="lower right")
plt.show()

We observe above that the AUC scores for all the 9 classes are very high, even though the classes in the original training dataset were highly imbalanced.

In [ ]:
# summarize class distribution
print (Counter(y_val_pred))

In [ ]:
plt.figure()
plt.title("Frequency Distribution of Predicted Classes for Validation Dataset")
sns.countplot(y_val_pred)
plt.xlabel("Predicted Class");

In [ ]:
# Print classification report
target_names = ['class 1', 'class 2', 'class 3', 'class 4', 'class 5',
                'class 6', 'class 7', 'class 8', 'class 9']
print(classification_report(y_val, y_val_pred, target_names=target_names))

We now display the balanced accuracy based on the validation dataset, which can cater to imbalanced datasets. It is the macro-average of recall scores per class.

## Evaluation of Alternative Model vs Baseline Model

<div class="alert alert-block alert-info">
    
(To be written)
    
</div>

## Data Export (for Kaggle Submission)

In [ ]:
y_test_pred = alternative_clf.predict(X_test)

In [ ]:
y_test_pred.shape

In [ ]:
# Restore the 'id' column since we need this for the Kaggle submission
test_pred = pd.concat([test['id'], pd.DataFrame(y_test_pred, columns=['class'])], axis=1)

In [ ]:
# Verify that we have a mix of predictions for variation classes
test_pred['class'].value_counts()

In [ ]:
test_pred.head()

In [ ]:
# We export the predictions
test_pred.to_csv("../assets/test_pred.csv", index=False)